In [ ]:
# React_agent

In [1]:
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict, Callable


@dataclass
class Step:
    thought: str
    action: Optional[Tuple[str, str]] = None  # (tool_name, arguments)
    observation: Optional[str] = None


class ReActAgent:
    def __init__(self, tools: Dict[str, Callable], max_steps: int = 8):
        self.tools = tools
        self.max_steps = max_steps
        self.trace: List[Step] = []

    def _log(self, label: str, msg: str):
        print(f"[{label}] {msg}")

    def run(self, query: str) -> str:
        self._log("Consulta", query)

        # Paso 1: pensar sobre la descomposición
        s1 = Step(thought="Necesito sumar (7 + 5). Luego multiplicar el resultado por 3.")
        self.trace.append(s1)
        self._log("Pensamiento", s1.thought)

        # Paso 2: acción con herramienta 'calculator'
        s1.action = ("calculator", "7+5")
        self._log("Acción", f"{s1.action[0]}('{s1.action[1]}')")
        obs1 = str(self.tools["calculator"]("7+5"))
        s1.observation = obs1
        self._log("Observación", obs1)

        # Paso 3: siguiente pensamiento
        s2 = Step(thought=f"Ahora debo calcular 3 * {obs1}.")
        self.trace.append(s2)
        self._log("Pensamiento", s2.thought)

        # Paso 4: segunda acción
        expr2 = f"3*{obs1}"
        s2.action = ("calculator", expr2)
        self._log("Acción", f"{s2.action[0]}('{s2.action[1]}')")
        obs2 = str(self.tools["calculator"](expr2))
        s2.observation = obs2
        self._log("Observación", obs2)

        final = f"El resultado es {obs2}."
        self._log("Respuesta Final", final)

        return final


In [3]:
def calculator(expr: str) -> float:
    allowed = set("0123456789.+-*/() ")
    if any(ch not in allowed for ch in expr):
        raise ValueError("Expresión inválida.")
    return eval(expr)


TOOLS = {
    "calculator": calculator
}


In [ ]:
# Demo

In [4]:
agent = ReActAgent(tools=TOOLS, max_steps=8)
print(agent.run("¿Cuánto es 3 veces la suma de 7 y 5?"))


[Consulta] ¿Cuánto es 3 veces la suma de 7 y 5?
[Pensamiento] Necesito sumar (7 + 5). Luego multiplicar el resultado por 3.
[Acción] calculator('7+5')
[Observación] 12
[Pensamiento] Ahora debo calcular 3 * 12.
[Acción] calculator('3*12')
[Observación] 36
[Respuesta Final] El resultado es 36.
El resultado es 36.
